&emsp;&emsp;考虑盒子和球模型$\lambda=(A,B,\boldsymbol{\pi})$,状态集合$Q=\{1, 2, 3 \}$,观测集合$V=\{ 红,白\}$,

\begin{equation}
A=\left[\begin{array}{ccc}
0.5 & 0.2 & 0.3 \\
0.3 & 0.5 & 0.2 \\
0.2 & 0.3 & 0.5
\end{array}\right], \quad
B=\left[\begin{array}{cc}
0.5 & 0.5 \\
0.4 & 0.6 \\
0.7 & 0.3
\end{array}\right], \quad
\pi=\left[\begin{array}{c}
0.2 \\
0.4 \\
0.4
\end{array}\right]
\end{equation}

其中$T=3$,$O=\{红,白,红\}$

In [65]:
import numpy as np

# 初始化
A = np.array([[0.5, 0.2, 0.3],
              [0.3, 0.5, 0.2],
              [0.2, 0.3, 0.5]])
B = np.array([[0.5, 0.5],
              [0.4, 0.6],
              [0.7, 0.3]])

pi = np.array([[0.2, 0.4, 0.4]]).T

### 初值

\begin{array}{l}
\alpha_{1}(1)=\pi_{1} b_{1}\left(o_{1}\right)=0.10 \\
\alpha_{1}(2)=\pi_{2} b_{2}\left(o_{1}\right)=0.16 \\
\alpha_{1}(3)=\pi_{3} b_{3}\left(o_{1}\right)=0.28
\end{array}

In [66]:
# 计算初值,此时o_1 = 红
def init_value(pi,  # 初始状态概率向量
               B,  # 观测概率分布
               color):  # 此次观测的颜色
    """计算初值"""
    if color == '红':
        index = 0
    else:
        index = 1
    this_B = B[:, index].reshape(-1, 1)
    return pi * this_B
    pass


a_1 = init_value(pi, B, '红')  # 初始前向概率
a_1

array([[0.1 ],
       [0.16],
       [0.28]])

### 递推

\begin{array}{l}
\alpha_{2}(1)=\left[\sum_{i=1}^{3} \alpha_{1}(i) a_{i 1}\right] b_{1}\left(o_{2}\right)=0.154 \times 0.5=0.077 \\
\alpha_{2}(2)=\left[\sum_{i=1}^{3} \alpha_{1}(i) a_{i 2}\right] b_{2}\left(o_{2}\right)=0.184 \times 0.6=0.1104 \\
\alpha_{2}(3)=\left[\sum_{i=1}^{3} \alpha_{1}(i) a_{i 3}\right] b_{3}\left(o_{2}\right)=0.202 \times 0.3=0.0606 \\
\alpha_{3}(1)=\left[\sum_{i=1}^{3} \alpha_{2}(i) a_{i 1}\right] b_{1}\left(o_{3}\right)=0.04187 \\
\alpha_{3}(2)=\left[\sum_{i=1}^{3} \alpha_{2}(i) a_{i 2}\right] b_{2}\left(o_{3}\right)=0.03551 \\
\alpha_{3}(3)=\left[\sum_{i=1}^{3} \alpha_{2}(i) a_{i 3}\right] b_{3}\left(o_{3}\right)=0.05284
\end{array}

In [67]:
def recur_com(last_a,  # 上一时刻的前向概率
              A,  # 状态转移概率分布
              B,  # 观测概率分布
              color):  # 此次观测的颜色
    """前向概率的递推计算"""
    if color == '红':
        index = 0
    else:
        index = 1
    step1 = np.sum(last_a * A, axis=0).reshape(-1, 1)
    return step1 * B[:, index].reshape(-1, 1)

In [68]:
# 递推计算,此时o_2 = 白
a_2 = recur_com(a_1, A, B, color='白')
a_2

array([[0.077 ],
       [0.1104],
       [0.0606]])

In [69]:
# 递推计算,此时o_3=红
a_3 = recur_com(a_2, A, B, color='红')
a_3

array([[0.04187 ],
       [0.035512],
       [0.052836]])

### 终止

$$ P(O \mid \boldsymbol{\lambda})=\sum_{i=1}^{3} \alpha_{3}(i)=0.13022 $$

表示在马尔可夫模型参数为$\boldsymbol{\lambda}$,观测序列为$O$的概率

In [70]:
# 终止
P_O_lambda = sum(a_3)
float(P_O_lambda)  # 该马尔可夫模型下,观测序列为{红,白,红}的概率

0.130218